In [1]:
import pandas as pd
import numpy as np
import zipfile
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/instquality/")

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/instquality/")

In [3]:
from functions.load import *

datasets = load()

(efw, # economic freedom of the world, fraser institute, 1970-2021
 fiw, # freedom in the world, freedom house, 2006-2023
 ief, # index of economic freedom, heritage foundation, 1995-2023
 p5d, # polity5d, polity project, 1960-2020
 pts, # political terror scale, 1976-2022
 wgi, # world governance indicators, world bank, 1996-2022
 wb, # institutional quality data + macro data (control), world bank, 1960-2023
 tgt # 10y sovereign yields (target), OECD and world bank, 1960-2024

) = (datasets[k] for k in ["efw", "fiw", "ief", "p5d", "pts", "wgi", "wb", "tgt"])

Loading files/EFW.xlsx
Loading files/FIW.xlsx
Loading files/IEF.csv
Loading files/P5D.xls
Loading files/PTS.xlsx
Loading files/WGI.xlsx
Loading files/WB_DATA.zip
Loading files/OECD.csv


In [4]:
from functions.merge import *

dfs = [efw, fiw, ief, p5d, pts, wgi, wb, tgt]
names = ['efw', 'fiw', 'ief', 'p5d', 'pts', 'wgi', 'wb', 'tgt']

conc = create_concordance(dfs)

for i, name in enumerate(names):
    dfs[i] = apply_concordance(dfs[i], conc, name)

efw, fiw, ief, p5d, pts, wgi, wb, tgt = dfs

cdfs = [df.drop_duplicates(subset=['country', 'year'])
        for df in dfs
        #if df is not wb
]

merged = merge(cdfs, conc)

Removed from efw: []
Removed from fiw: ['Abkhazia' 'Antigua and Barbuda' 'Chechnya' 'Crimea' 'Eastern Donbas'
 'Gaza Strip' 'Indian Kashmir' 'Israeli Occupied Territories' 'Kiribati'
 'Marshall Islands' 'Micronesia' 'Nagorno-Karabakh' 'Northern Cyprus'
 'Pakistani Kashmir' 'Palau' 'Samoa' 'Solomon Islands' 'Somaliland'
 'South Ossetia' 'St. Kitts and Nevis' 'St. Lucia'
 'St. Vincent and the Grenadines' 'Tibet' 'Tonga' 'Transnistria' 'Tuvalu'
 'Vanuatu']
Removed from ief: ['Kiribati' 'Micronesia' 'Saint Lucia' 'Saint Vincent and the Grenadines'
 'Samoa' 'Solomon Islands' 'Tonga' 'Vanuatu']
Removed from p5d: ['Solomon Islands' 'South Vietnam' 'Vietnam North' 'Yemen North'
 'Yemen South']
Removed from pts: ['Antigua and Barbuda' 'Crimea' 'European Union' 'Gaza (Hamas)'
 'Israel in Occupied Territories' 'Israel in pre-1967 borders' 'Kiribati'
 'Marshall Islands' 'Micronesia, Federated States of' 'Palau'
 'Russia-Occupied Areas (Ukraine)' 'Saint Kitts and Nevis' 'Saint Lucia'
 'Saint Vincen

In [ ]:
# # dropping countries without target data

# print(int((merged[merged['tgt_yield'].notna()].country.nunique()/merged.country.nunique())*100//1), f"percent of countries have yield (target) data ({merged[merged['tgt_yield'].notna()].country.nunique()} countries)")

# remove = (
#     merged.groupby('country')['tgt_yield']
#     .apply(lambda x: x.notna().any())
#     .loc[lambda s: ~s]
#     .index
# )

# merged = merged[~merged['country'].isin(remove)].copy()

# print("Dropped countries without yield (target) data:", list(remove))

36 percent of countries have yield (target) data (72 countries)
Dropped countries without yield (target) data: ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Barbados', 'Belarus', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Cuba', 'Curacao', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Faroe Islands', 'French Guiana', 'French Polynesia', 'Gabon', 'Gambia', 'Georgia', 'Germany East', 'Gibraltar', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong SAR, China', 'Iran', 'Iraq', 'Jamaica', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait', 'Laos', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', '

In [5]:
from functions.fill import *

df = fill(merged)

F-filling Afghanistan values
F-filling Albania values
F-filling Algeria values
F-filling Andorra values
F-filling Angola values
F-filling Argentina values
F-filling Armenia values
F-filling Australia values
F-filling Austria values
F-filling Azerbaijan values
F-filling Bahamas values
F-filling Bahrain values
F-filling Bangladesh values
F-filling Barbados values
F-filling Belarus values
F-filling Belgium values
F-filling Belize values
F-filling Benin values
F-filling Bermuda values
F-filling Bhutan values
F-filling Bolivia values
F-filling Bosnia and Herzegovina values
F-filling Botswana values
F-filling Brazil values
F-filling Brunei Darussalam values
F-filling Bulgaria values
F-filling Burkina Faso values
F-filling Burundi values
F-filling Cabo Verde values
F-filling Cambodia values
F-filling Cameroon values
F-filling Canada values
F-filling Central African Republic values
F-filling Chad values
F-filling Chile values
F-filling China values
F-filling Colombia values
F-filling Comoros v

In [6]:
# save

df.to_pickle("files/inst.dat")